In [2]:
# warning 무시
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
# glob : 파일 리스트 뽑기
from glob import glob

# tqdm : for문 진행률 확인
from tqdm import tqdm

from collections import deque

In [4]:
# requests : HTTP를 사용하기 위해 쓰여지는 라이브러리
import requests

# multiprocessing : process spawing
import multiprocessing
import pandas as pd
import numpy as np
import os

# 질문
flag : 수집되는 거 안되는 거 구분? 

recursive = True 용도? 

df['num_lang'] = df['languages'].fillna('').apply(lambda x: len(x.split())) 결측치 공백 입력, x : df['languages'].fillna('') 인가요? 

df.to_csv(f'{steamspy_dir}\\{file_name}.csv', index = False, sep = '\t')
    index = False?

In [6]:
cwd = os.getcwd() # 로컬에서 파일을 가져오려고 cwd -> 같은 위치에 있는 상대경로 활용
# c:\Users\NT550009\Desktop\ASAC_ml_project\web_crawling\JC\2_steampy_tag

check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive = True)[0]
# C:\Users\NT550009\Desktop\ASAC_ml_project\web_crawling\JC\2_steampy_tag\steamspy.txt

c:\Users\NT550009\Desktop\ASAC_ml_project\web_crawling\JC\2_steampy_tag
c:\Users\NT550009\Desktop\ASAC_ml_project\web_crawling\JC\2_steampy_tag\steamspy.txt


In [7]:
def extract(appid):
    print("완료")
    flag = ''
    cwd = os.getcwd()  # 로컬에서 파일을 가져오려고 cwd -> 같은 위치에 있는 상대경로 활용
    # steamspy.txt로 되있는 파일경로 찾아서 리스트로 출력은 ['steamspy.txt의 경로']
    check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive=True)[0]  # \\**\\ 경로 설정을 편하게. 성현님 코드에서 txt와 ipynb 가 다른 곳에 있음.


    # steamspy.txt에서 모을 내용 리스트 원소로 정리
    # steampy.txt를 줄 기준으로 읽어 온뒤 \n(엔터키)를 제거 open(check_path).readlines()의 출력은 ['1.~~~~\n', '2.~~~~\n' .....] 이런식
    # 결과적으로 check_list = ['1.~~~~', '2.~~~~' .....]
    check_list = [i.replace('\n', '') for i in open(check_path).readlines()]

    # appid 당 check_list 원소 정리
    # result_dict = {appid: {'1.~~~~':'', '2.~~~~': '', ........}}
    result_dict = {appid: {i: '' for i in check_list}}

    # sample : https://steamspy.com/api.php?request=appdetails&appid=1794680
    res = requests.get(f'https://steamspy.com/api.php?request=appdetails&appid={appid}')
    print("완료")
    # 요청이 성공적으로 수행되지 않았을 때
    if res.status_code != 200:  # status_code : 응답상태 확인
        flag = False
        return result_dict, flag

    # json 형식 dict으로 가져오기
    res_json = res.json()
    print("완료")
    for check in check_list:
        # 예외 처리
        # res_json 에서 check 키에 내용 찾아서 추가
        # result_dict = {appid: {'1.~~~~': res_json[check], ........}}
        try:
            result_dict[appid][check] = res_json[check]
        except:
            continue

    flag = True

    return result_dict, flag
    print("완료")



In [8]:
def run():
    cwd = os.getcwd()
    pwd = os.path.abspath(os.path.join(cwd, '..'))
    # 1_appid라는 이름이 포함되있는 txt 파일경로를 리스트로 가져옴(recursive=True로 하고 ** 쓰면 하위폴더까지 전부 검색)
    appid_dirs = glob(f'{pwd}\\**\\1_appid\\*.txt', recursive=True)
    # steamspy.txt로 되있는 파일경로 찾아서 리스트로 출력은 ['steamspy.txt의 파일 경로'] (하위 폴더까지 전부 검색)
    check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive=True)[0]
    
    # 이게 빠졌던거 같은데.. 뭐지???
    steamspy_dir = glob(f'{cwd}\\**\\steampy',recursive=True)[0]

    # steampy.txt를 줄 기준으로 읽어 온뒤 \n(엔터키)를 제거 open(check_path).readlines()의 출력은 ['1.~~~~\n', '2.~~~~\n' .....] 이런식
    check_list = [i.replace('\n', '') for i in open(check_path).readlines()]

    for appid_dir in appid_dirs:
        # appid_dirs 리스트 안에 있는 appid_dir들의 파일 이름만 추출
        base_name = os.path.basename(appid_dir)

        # 파일 이름 안에 2019 혹은 2018 이 있으면 제외
        if '2019' in base_name or '2018' in base_name:  # 2018, 2019 pass
            continue

        # appid_dir.txt를 줄 기준으로 읽어 온뒤 \n(엔터키)를 제거 open(appid_dir).readlines()의 출력은 ['~~~~\n', '~~~~\n' .....] 이런식
        app_list = [i.replace('\n', '') for i in open(appid_dir).readlines()]

        # 딕셔너리 컴프리헨션
        result_last = {i: deque([]) for i in check_list}  # {'developer': deque([]), 'publisher': deque([]) } 형식
        result_last["appid"] = app_list

        result_mid = {}
        result_error = {}

        # app_list로 extract()함수 반복
        for i in app_list:
            a = extract(i)
            print(a)
        '''
        with multiprocessing.Pool() as p: # 병렬처리
            for val, flag in tqdm(p.imap(extract, app_list, 8), total = len(app_list)):
                result_mid.update(val)
                if flag == False:
                    result_error.update(val)
        '''
        for app in app_list:
            for check in check_list:
                result_last[check].append(result_mid[app][check])

        df = pd.DataFrame(result_last)

        # 결측치 처리
        df['price'] = df['price'].fillna(0)
        df['price'] = df['price'].astype(int) / 100

        df['initialprice'] = df['initialprice'].fillna(0)
        df['initialprice'] = df['initialprice'].astype(int) / 100


        df['num_lang'] = df['languages'].fillna('').apply(lambda x: len(x.split()))

        file_name = os.path.splitext(base_name)[0]

        ## steamspy_dir 변수는 어디로 사라진건지 모르겠어
        df.to_csv(f'{steamspy_dir}\\{file_name}.csv', index=False, sep='\t')


if __name__ == '__main__':
    run()

glob(f'{cwd}\\**\\steamspy.txt', recursive=True)[0]

완료
완료
완료
({'1145360': {'developer': 'Supergiant Games', 'publisher': 'Supergiant Games', 'positive': 222501, 'negative': 3355, 'average_forever': 1754, 'average_2weeks': 786, 'median_forever': 954, 'median_2weeks': 764, 'price': '2499', 'initialprice': '2499', 'languages': 'English, French, Italian, German, Spanish - Spain, Korean, Portuguese - Brazil, Russian, Simplified Chinese, Polish, Japanese', 'tags': {'Action Roguelike': 1069, 'Rogue-lite': 740, 'Hack and Slash': 738, 'Indie': 721, 'Action': 685, 'Mythology': 634, 'Rogue-like': 561, 'RPG': 560, 'Great Soundtrack': 545, 'Singleplayer': 523, 'Dungeon Crawler': 507, 'Story Rich': 504, 'Replay Value': 443, 'Isometric': 338, 'Difficult': 311, 'Action RPG': 245, 'Hand-drawn': 233, 'Atmospheric': 220, 'Perma Death': 87, 'Souls-like': 80}, 'genre': 'Action, Indie, RPG'}}, True)
완료
완료
완료
({'546560': {'developer': 'Valve', 'publisher': 'Valve', 'positive': 78985, 'negative': 1276, 'average_forever': 559, 'average_2weeks': 76, 'median_fore

KeyError: '1145360'


    # 이게 빠졌던거 같은데.. 뭐지???
    steamspy_dir = glob(f'{cwd}\\**\\steampy',recursive=True)[0]


In [ ]:
check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive = True)[0]
check_list = [i.replace('\n','') for i in open(check_path).readlines()]
result_last= {i : deque([]) for i in check_list} 
print(result_last)

{'developer': deque([]), 'publisher': deque([]), 'positive': deque([]), 'negative': deque([]), 'average_forever': deque([]), 'average_2weeks': deque([]), 'median_forever': deque([]), 'median_2weeks': deque([]), 'price': deque([]), 'initialprice': deque([]), 'languages': deque([]), 'tags': deque([]), 'genre': deque([])}
